In [ ]:
!pip install celluloid
!pip install -U imgaug imageio
!pip install pytorch-lightning torch torchvision torchmetrics nibabel tqdm opencv-python
!pip install onedrivedownloader


In [ ]:
from onedrivedownloader import download

url = "https://ujchmura-my.sharepoint.com/:u:/g/personal/marcin_sztukowski_student_uj_edu_pl/EURfIwcEpYVKv88gopAwaeoBbNeeBOqWz3z4IkzUaI5uqg?e=aaQSj9"
download(url, filename="Task06_Lung.tar", unzip=False)
!tar -xvf /content/Task06_Lung.tar -C /content/

In [ ]:
%matplotlib notebook
from pathlib import Path
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from tqdm.notebook import tqdm
import cv2
import numpy as np


In [ ]:
from pathlib import Path

# Ustal folder projektu względem lokalizacji notebooka
project_root = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

# Ścieżki względne
root = project_root / "Task06_Lung" / "imagesTr"
label = project_root / "Task06_Lung" / "labelsTr"

print("Images path:", root)
print("Labels path:", label)
print("Images path exists:", root.exists())
print("Labels path exists:", label.exists())

**Task: Load a sample NIfTI and its corresponding label mask**<br />
Hint: You might want to define a helper function to make your life easier

In [ ]:
def change_img_to_label_path(path):
    """
    Replaces imagesTr with labelsTr
    """
    parts = list(path.parts)  # get all directories whithin the path
    parts[parts.index("imagesTr")] = "labelsTr"  # Replace imagesTr with labelsTr
    return Path(*parts)  # Combine list back into a Path object


In [ ]:

from pathlib import Path

root = Path("Task06_Lung")

# pobierz ścieżki do obrazów i masek
images = sorted((root / "imagesTr").glob("lung_*.nii.gz"))
labels = sorted((root / "labelsTr").glob("lung_*.nii.gz"))

# wybierz przykładowy plik (np. 2)
sample_path = images[2]                       # obraz
sample_path_label = labels[2]                 # odpowiadająca etykieta

print("Sample image path:", sample_path)
print("Sample label path:", sample_path_label)


In [ ]:
sample_path, sample_path_label

Load NIfTI and extract image data

In [ ]:
data = nib.load(sample_path)
label = nib.load(sample_path_label)

ct = data.get_fdata()
mask = label.get_fdata()

**Task: Find out the orientation**

In [ ]:
nib.aff2axcodes(data.affine)

In [ ]:
fig = plt.figure()
camera = Camera(fig)  # create the camera object from celluloid

for i in range(0, ct.shape[2], 2):  # axial view
    plt.imshow(ct[:,:,i], cmap="bone")
    mask_ = np.ma.masked_where(mask[:,:,i]==0, mask[:,:,i])
    plt.imshow(mask_, alpha=0.5, cmap="autumn")
    #plt.axis("off")
    camera.snap()  # Store the current slice



# Tworzenie i zapis animacji
animation = camera.animate(interval=100)
animation.save("/content/ct_mask_animation_raw.mp4", writer="ffmpeg")

# --- 🎬 wyświetl w notebooku ---
from IPython.display import Video
Video('/content/ct_mask_animation_raw.mp4', embed=True)


## Preprocessing
**Task: Please perform the following preprocessing steps:**

1. CT images have a fixed range from -1000 to 3071. **Thus you could normalize by dividing by 3071** <br /> You don't need to compute mean and standard deviation for this task
2. As we want to focus on lung tumors, we can crop away parts of the lower abdomen to reduce the complexity and help the network learn. As an example, **you might skip the first 30 slices (from lower abdomen to the neck)** (last axis)
3. As we want to tackle this task on a slice level (2D) and not on a subject level (3D) to reduce the computational cost **you should store the preprocessed data as 2d files**, because reading a single slice is much faster than loading the complete NIfTI file.
4. Resize the single slices and masks to (256, 256) (when resizing the mask, pass interpolation=cv2.INTER_NEAREST to the resize function to apply nearest neighbour interpolation)

Loop over all_files and apply the preprocessing steps. <br />

Additionally, please make sure that all scans have the same orientation

In the preprocessing loop, you need to create a directory for each subject containg the ct and label slices with identical names. <br />
E.g:
* 0/data/0.npy
* 0/masks/0.npy

Store the last 6 subjects as validation data

PS: Feel free to try the lung window!

In [ ]:
!find /content/Task06_Lung -name "._*" -type f -delete

In [ ]:
from pathlib import Path
import nibabel as nib
import cv2
import numpy as np
from tqdm import tqdm

root = Path("/content/Task06_Lung")
save_root = root / "Preprocessed"

all_files = sorted((root / "imagesTr").glob("*.nii.gz"))

for counter, path_to_ct_data in enumerate(tqdm(all_files)):
    path_to_label = root / "labelsTr" / path_to_ct_data.name

    ct_data = nib.load(path_to_ct_data).get_fdata().astype(np.float32)
    label_data = nib.load(path_to_label).get_fdata().astype(np.float32)

    ct_data = ct_data[:, :, 30:] / 3071.0
    label_data = label_data[:, :, 30:]

    if counter < len(all_files) - 6:
        current_path = save_root / "train" / str(counter)
    else:
        current_path = save_root / "val" / str(counter)

    for i in range(ct_data.shape[-1]):
        slice_ = cv2.resize(ct_data[:, :, i], (256, 256))
        mask_ = cv2.resize(label_data[:, :, i], (256, 256), interpolation=cv2.INTER_NEAREST)

        (current_path / "data").mkdir(parents=True, exist_ok=True)
        (current_path / "masks").mkdir(parents=True, exist_ok=True)

        np.save(current_path / "data" / f"{i}.npy", slice_)
        np.save(current_path / "masks" / f"{i}.npy", mask_)

## Validate preprocessed data
**Task: Take a look at your stored files and inspect if everything worked as expected**

In [ ]:
path = Path("Task06_Lung/Preprocessed/train/2")  # Select a subject. Check the folder if it exists


In [ ]:

list(path.glob("*"))

# Choose a file and load slice + mask
file = "120.npy"
slice = np.load(path/"data"/file)
mask = np.load(path/"masks"/file)


In [ ]:
from pathlib import Path
import numpy as np

base_path = Path("/content/Task06_Lung/Preprocessed/train")

max_pixels = 0
best_subject = None
best_slice_idx = None

for subj in sorted(base_path.glob("*")):
    for mask_file in (subj / "masks").glob("*.npy"):
        mask = np.load(mask_file)
        tumor_pixels = np.sum(mask > 0)
        if tumor_pixels > max_pixels:
            max_pixels = tumor_pixels
            best_subject = subj.name
            best_slice_idx = mask_file.stem  # npy filename without .npy

print(f"📍 Największy guz w: {best_subject}/slice {best_slice_idx}")
print(f"🩸 Liczba pikseli guza: {max_pixels}")



import matplotlib.pyplot as plt
from IPython.display import Image, display

subject = best_subject
slice_idx = best_slice_idx

# --- wczytaj dane ---
slice_ = np.load(f"/content/Task06_Lung/Preprocessed/train/{subject}/data/{slice_idx}.npy")
mask = np.load(f"/content/Task06_Lung/Preprocessed/train/{subject}/masks/{slice_idx}.npy")

# --- normalizacja i RGB overlay ---
slice_norm = (slice_ - np.min(slice_)) / (np.max(slice_) - np.min(slice_) + 1e-8)
slice_uint8 = (slice_norm * 255).astype(np.uint8)
mask_uint8 = (mask * 255).astype(np.uint8)

rgb = np.stack([slice_uint8]*3, axis=-1)
overlay = rgb.copy()
overlay[mask_uint8 > 0, 0] = 255  # Red
overlay[mask_uint8 > 0, 1] = 0
overlay[mask_uint8 > 0, 2] = 0

# --- wizualizacja ---
fig, axis = plt.subplots(1, 2, figsize=(8, 8))
axis[0].imshow(slice_uint8, cmap="gray")
axis[0].set_title(f"CT slice {slice_idx}")

axis[1].imshow(overlay)
axis[1].set_title("Overlay (max tumor area)")

plt.tight_layout()
save_path = f"/content/ct_mask_largest_overlay.png"
plt.savefig(save_path, bbox_inches="tight", dpi=150)
plt.close(fig)

display(Image(filename=save_path))

In [ ]:
print("slice shape:", slice.shape)
print("mask shape:", mask.shape)
print("unique mask values:", np.unique(mask))
print(slice.min(), slice.max())

## Introduction
In this notebook we will create the dataset class used to feed slices and corresponding segmentation masks to the network during training.
It is identical to the CardiacDataset

In [ ]:
# --- Fix for NumPy 2.0 removal of np.sctypes ---
import numpy as np

if not hasattr(np, "sctypes"):
    np.sctypes = {
        'int': [np.int8, np.int16, np.int32, np.int64],
        'uint': [np.uint8, np.uint16, np.uint32, np.uint64],
        'float': [np.float16, np.float32, np.float64],
        'complex': [np.complex64, np.complex128],
        'others': [bool, object, str, bytes],
    }
# ------------------------------------------------

In [ ]:
from pathlib import Path
import torch
import numpy as np
import imgaug
import imgaug.augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage
import matplotlib.pyplot as plt



## DataSet Creation
We need to implement the following functionality:
1. Create a list of all 2D slices. To so we need to extract all slices from all subjects
2. Extract the corresponding label path for each slice path
3. Load slice and label
4. Data Augmentation. Make sure that slice and mask are augmented identically. imgaug handles this for us, thus we will not use torchvision.transforms for that
5. Return slice and mask

In [ ]:
class LungDataset(torch.utils.data.Dataset):
    def __init__(self, root, augment_params):
        self.all_files = self.extract_files(root)
        self.augment_params = augment_params

    @staticmethod
    def extract_files(root):
        """
        Extract the paths to all slices given the root path (ends with train or val)
        """
        files = []
        for subject in root.glob("*"):   # Iterate over the subjects
            slice_path = subject/"data"  # Get the slices for current subject
            for slice in slice_path.glob("*"):
                files.append(slice)
        return files


    @staticmethod
    def change_img_to_label_path(path):
        """
        Replace data with mask to get the masks
        """
        parts = list(path.parts)
        parts[parts.index("data")] = "masks"
        return Path(*parts)

    def augment(self, slice, mask):
        """
        Augments slice and segmentation mask in the exact same way
        Note the manual seed initialization
        """
        ###################IMPORTANT###################
        # Fix for https://discuss.pytorch.org/t/dataloader-workers-generate-the-same-random-augmentations/28830/2
        random_seed = torch.randint(0, 1000000, (1,))[0].item()
        imgaug.seed(random_seed)
        #####################################################

        # mask = SegmentationMapsOnImage(mask, mask.shape) wazna poprawka na dole
        mask = SegmentationMapsOnImage(mask.astype(np.uint8), mask.shape)

        slice_aug, mask_aug = self.augment_params(image=slice, segmentation_maps=mask)
        mask_aug = mask_aug.get_arr()
        return slice_aug, mask_aug

    def __len__(self):
        """
        Return the length of the dataset (length of all files)
        """
        return len(self.all_files)


    def __getitem__(self, idx):
        """
        Given an index return the (augmented) slice and corresponding mask
        Add another dimension for pytorch
        """
        file_path = self.all_files[idx]
        mask_path = self.change_img_to_label_path(file_path)
        slice = np.load(file_path)
        mask = np.load(mask_path)

        if self.augment_params:
            slice, mask = self.augment(slice, mask)

        return np.expand_dims(slice, 0), np.expand_dims(mask, 0)


Now we can test our dataset!
Let us at first define the data augmentation routine corresponding of scaling and rotation

In [ ]:
seq = iaa.Sequential([
    iaa.Affine(scale=(0.85, 1.15), # zoom in or out
               rotate=(-45, 45)),  # rotate up to 45 degrees
    iaa.ElasticTransformation()
                ])


In [ ]:
# Create the dataset object
path = Path("Task06_Lung/Preprocessed/train/")
dataset = LungDataset(path, seq)

Finally we can visualize our dataset.
To make sure that the augmentation works we access the same element multiple times and check whether the segmentation masks fit

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display

# --- Rysuj 3x3 przykłady augmentacji ---
fig, axis = plt.subplots(3, 3, figsize=(9, 9))

for i in range(3):
    for j in range(3):
        slice, mask = dataset[19]
        mask_ = np.ma.masked_where(mask == 0, mask)

        axis[i][j].imshow(slice[0], cmap="bone")
        axis[i][j].imshow(mask_[0], cmap="autumn", alpha=0.5)
        axis[i][j].axis("off")

fig.suptitle("Sample augmentations")
plt.tight_layout()

# --- 💾 Zapisz do pliku PNG ---
save_path = "/content/sample_augmentations.png"
plt.savefig(save_path, bbox_inches="tight", dpi=150)
plt.close(fig)

# --- 🖼️ Wyświetl zapisany obraz ---
display(Image(filename=save_path))

Nice!
With above dataset we can finally create the model and train the Lung Cancer Segmentation Model

## Introduction
In this notebook we will create the model for the Tumor Segmentation! <br />
Nothing is changed compared to the Atrium Segmentation


In [ ]:
import torch

## UNET
The idea behind a UNET is that we have "Downconvolutions" which are reducing the size of the image combined with increasing filter size followed by "Upconvolutions" which increase the image size up to the original size while reducing the number of filters. <br />
All pairs between Up- and Downconvolutions are linked with skip connections.<br />
Upsampling can either be done by interpolation or by UpConvolutions (ConvTranspose2d)




## Convolutions
At first we write the class, repsonsible for the convolutions.
We will use two convolutions between each down- or upconvolution step

In [ ]:
class DoubleConv(torch.nn.Module):
    """
    Helper Class which implements the intermediate Convolutions
    """
    def __init__(self, in_channels, out_channels):

        super().__init__()
        self.step = torch.nn.Sequential(torch.nn.Conv2d(in_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU(),
                                        torch.nn.Conv2d(out_channels, out_channels, 3, padding=1),
                                        torch.nn.ReLU())

    def forward(self, X):
        return self.step(X)


## UNET
With the help of DoubleConv we can easily implement the UNET by combining **DoubleConv** with **maxpooling** for  DownConvolutions or **DoubleConv** with **Upsample** for UpConv.

Feel free to replace Upsample with ConvTranspose2d

In [ ]:
class UNet(torch.nn.Module):
    """
    This class implements a UNet for the Segmentation
    We use 3 down- and 3 UpConvolutions and two Convolutions in each step
    """

    def __init__(self):
        """Sets up the U-Net Structure
        """
        super().__init__()


        ############# DOWN #####################
        self.layer1 = DoubleConv(1, 64)  # MRI -> One channel
        self.layer2 = DoubleConv(64, 128)
        self.layer3 = DoubleConv(128, 256)
        self.layer4 = DoubleConv(256, 512)

        #########################################

        ############## UP #######################
        self.layer5 = DoubleConv(512 + 256, 256)
        self.layer6 = DoubleConv(256+128, 128)
        self.layer7 = DoubleConv(128+64, 64)
        self.layer8 = torch.nn.Conv2d(64, 1, 1)  # Binary label -> One class
        #########################################

        self.maxpool = torch.nn.MaxPool2d(2)

    def forward(self, x):

        ####### DownConv 1#########
        x1 = self.layer1(x)
        x1m = self.maxpool(x1)
        ###########################

        ####### DownConv 2#########
        x2 = self.layer2(x1m)
        x2m = self.maxpool(x2)
        ###########################

        ####### DownConv 3#########
        x3 = self.layer3(x2m)
        x3m = self.maxpool(x3)
        ###########################

        ##### Intermediate Layer ##
        x4 = self.layer4(x3m)
        ###########################

        ####### UpCONV 1#########
        x5 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x4)  # Upsample with a factor of 2
        #x5 = torch.nn.ConvTranspose2d(512, 512, 2, 2)(x4)
        x5 = torch.cat([x5, x3], dim=1)  # Skip-Connection
        x5 = self.layer5(x5)
        ###########################

        ####### UpCONV 2#########
        x6 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x5)
        #x6 = torch.nn.ConvTranspose2d(256, 256, 2, 2)(x5)
        x6 = torch.cat([x6, x2], dim=1)  # Skip-Connection
        x6 = self.layer6(x6)
        ###########################

        ####### UpCONV 3#########
        x7 = torch.nn.Upsample(scale_factor=2, mode="bilinear")(x6)
        #x7 = torch.nn.ConvTranspose2d(128, 128, 2, 2)(x6)
        x7 = torch.cat([x7, x1], dim=1)
        x7 = self.layer7(x7)
        ###########################

        ####### Predicted segmentation#########
        ret = self.layer8(x7)
        return ret

## Testing

In [ ]:
model = UNet()

In [ ]:
random_input = torch.randn(1, 1, 256, 256)
output = model(random_input)
assert output.shape == torch.Size([1, 1, 256, 256])

## Introduction
Finally we are going to train our tumor segmentation network. <br />
Here we apply some small changes

In [ ]:
from pathlib import Path
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import imgaug.augmenters as iaa
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from celluloid import Camera

#from dataset import LungDataset
#from model import UNet

## Dataset Creation
Here we create the train and validation dataset. <br />
Additionally we define our data augmentation pipeline.
Subsequently the two dataloaders are created

In [ ]:
seq = iaa.Sequential([
    iaa.Affine(translate_percent=(0.15),
               scale=(0.85, 1.15), # zoom in or out
               rotate=(-45, 45)#
               ),  # rotate up to 45 degrees
    iaa.ElasticTransformation()  # Elastic Transformations
                ])


In [ ]:
# Create the dataset objects
train_path = Path("Task06_Lung/Preprocessed/train/")
val_path = Path("Task06_Lung/Preprocessed/val/")

train_dataset = LungDataset(train_path, seq)
val_dataset = LungDataset(val_path, None)

print(f"There are {len(train_dataset)} train images and {len(val_dataset)} val images")

## Oversampling to tackle strong class imbalance
Lung tumors are often very small, thus we need to make sure that our model does not learn a trivial solution which simply outputs 0 for all voxels.<br />
In this notebook we will use oversampling to sample slices which contain a tumor more often.

To do so we can use the **WeightedRandomSampler** provided by pytorch which needs a weight for each sample in the dataset.
Typically you have one weight for each class, which means that we need to calculate two weights, one for slices without tumors and one for slices with a tumor and create list that assigns each sample from the dataset the corresponding weight

To do so, we at first need to create a list containing only the class labels:

In [ ]:
target_list = []
for _, label in tqdm(train_dataset):
    # Check if mask contains a tumorous pixel:
    if np.any(label):
        target_list.append(1)
    else:
        target_list.append(0)

In [ ]:

target_array = np.array(target_list)
print(f"🔍 Łącznie: {len(target_array)} slice’ów")
print(f"🧬 Z guzem: {np.sum(target_array)}")
print(f"⚪ Bez guza: {len(target_array) - np.sum(target_array)}")
print(f"📊 Procent z guzem: {100*np.mean(target_array):.2f}%")

Then we can calculate the weight for each class:
To do so, we can simply compute the fraction between the classes and then create the weight list

In [ ]:
uniques = np.unique(target_list, return_counts=True)
uniques

In [ ]:
fraction = uniques[1][0] / uniques[1][1]
fraction

Subsequently we assign the weight 1 to each slice without a tumor and ~9 to each slice with a tumor

In [ ]:
weight_list = []
for target in target_list:
    if target == 0:
        weight_list.append(1)
    else:
        weight_list.append(fraction)

Finally we create the sampler which we can pass to the DataLoader.
**Important:** Only use a sampler for the train loader! We don't want to change the validation data to get a real validation

In [ ]:
sampler = torch.utils.data.sampler.WeightedRandomSampler(weight_list, len(weight_list))


In [ ]:
batch_size = 16#TODO
num_workers = 6# TODO


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                           num_workers=num_workers, sampler=sampler)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False)


We can verify that our sampler works by taking a batch from the train loader and count how many labels are larger than zero

In [ ]:
verify_sampler = next(iter(train_loader))  # Take one batch

In [ ]:
(verify_sampler[1][:,0]).sum([1, 2]) > 0  # ~ half the batch size

## Loss

As this is a harder task to train you might try different loss functions:
We achieved best results by using the Binary Cross Entropy instead of the Dice Loss

## Full Segmentation Model

We now combine everything into the full pytorch lightning model

In [ ]:
class TumorSegmentation(pl.LightningModule):
    def __init__(self):
        super().__init__()

        self.model = UNet()

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

    def forward(self, data):
        pred = self.model(data)
        return pred

    def training_step(self, batch, batch_idx):
        ct, mask = batch
        mask = mask.float()

        pred = self(ct)
        loss = self.loss_fn(pred, mask)

        # Logs
        self.log("Train_Dice", loss)
        if batch_idx % 50 == 0:
            self.log_images(ct.cpu(), pred.cpu(), mask.cpu(), "Train")
        return loss


    def validation_step(self, batch, batch_idx):
        ct, mask = batch
        mask = mask.float()

        pred = self(ct)
        loss = self.loss_fn(pred, mask)

        # Logs
        self.log("Val_Dice", loss)
        if batch_idx % 50 == 0:
            self.log_images(ct.cpu(), pred.cpu(), mask.cpu(), "Val")

        return loss


    def log_images(self, ct, pred, mask, name):

        results = []

        pred = pred > 0.5 # As we use the sigomid activation function, we threshold at 0.5


        fig, axis = plt.subplots(1, 2)
        axis[0].imshow(ct[0][0], cmap="bone")
        mask_ = np.ma.masked_where(mask[0][0]==0, mask[0][0])
        axis[0].imshow(mask_, alpha=0.6)
        axis[0].set_title("Ground Truth")

        axis[1].imshow(ct[0][0], cmap="bone")
        mask_ = np.ma.masked_where(pred[0][0]==0, pred[0][0])
        axis[1].imshow(mask_, alpha=0.6, cmap="autumn")
        axis[1].set_title("Pred")

        self.logger.experiment.add_figure(f"{name} Prediction vs Label", fig, self.global_step)



    def configure_optimizers(self):
        #Caution! You always need to return a list here (just pack your optimizer into one :))
        return [self.optimizer]



In [ ]:
# Instanciate the model
model = TumorSegmentation()

In [ ]:


# --- CALLBACK: zapisywanie najlepszych modeli ---
checkpoint_callback = ModelCheckpoint(
    dirpath="./checkpoints",         # katalog na checkpointy
    filename="epoch{epoch:02d}-valdice{Val_Dice:.4f}",  # nazwa pliku z metryką
    monitor="Val_Dice",              # 👈 musi być taka sama jak nazwa w self.log()
    save_top_k=3,                    # np. tylko 3 najlepsze checkpointy
    mode="min",                      # jeśli loss/Dice — zwykle "min"; jeśli accuracy — "max"
    save_last=True,                  # zapisuj też ostatni model
    verbose=True
)

# --- LOGGER: TensorBoard ---
logger = TensorBoardLogger(save_dir="./logs", name="lung_unet")

# --- TRENER: GPU, logowanie, callbacki ---
trainer = pl.Trainer(
    accelerator="gpu",     # użyj GPU (Colab T4)
    devices=1,             # tylko jeden GPU
    logger=logger,
    callbacks=[checkpoint_callback],
    max_epochs=20,
    log_every_n_steps=10,  # loguj co 10 batchy (nie co batch)
)

In [ ]:
trainer.fit(model, train_loader, val_loader)

## Evaluation


# Dice Score — Evaluation Metric for Medical Image Segmentation

The **Dice Score** (also called *Dice Coefficient* or *Sørensen–Dice index*)
is one of the most common evaluation metrics in **medical image segmentation**,
especially for CT or MRI scans where the model (e.g. **U-Net**) detects tumors or organs.

---

### Mathematical Definition

For two binary masks — prediction \( P \) and ground truth \( G \):

$$
\text{Dice}(P, G) = \frac{2 |P \cap G|}{|P| + |G|}
$$

where:
- $ |P \cap G| $ – number of pixels correctly predicted as tumor,
- $ |P| $ – number of pixels predicted as tumor,
- $ |G| $ – number of pixels actually belonging to tumor.

---

###  Practical Meaning

| Dice Score | Interpretation |
|-------------|----------------|
| **1.0** | Perfect overlap (model segmentation = ground truth) |
| **0.0** | No overlap (model completely wrong) |
| **0.5–0.9** | Partial overlap — typical for real segmentation tasks |

> 💡 Dice Score is very sensitive to small regions —
> if tumors are tiny, even a few wrong pixels can drastically reduce the score.


---

### How to compute Dice Score

```python
import torch

# Compute overall Dice Score
dice_score = DiceScore()(
    torch.from_numpy(preds),
    torch.from_numpy(labels).unsqueeze(0).float()
)
print(f"The Val Dice Score is: {dice_score:.4f}")
```
---


In [ ]:
class DiceScore(torch.nn.Module):
    """
    class to compute the Dice Loss
    """
    def __init__(self):
        super().__init__()

    def forward(self, pred, mask):

        #flatten label and prediction tensors
        pred = torch.flatten(pred)
        mask = torch.flatten(mask)

        counter = (pred * mask).sum()  # Counter
        denum = pred.sum() + mask.sum()  # denominator
        dice = (2*counter)/denum

        return dice


In [ ]:
from onedrivedownloader import download
url = "https://ujchmura-my.sharepoint.com/:u:/g/personal/marcin_sztukowski_student_uj_edu_pl/EeIxSRDsbaROj51n9jl7TDoBTifXVohnxCVymwhyiGrBSQ?e=IRznML"
download(url, filename="epoch=29-step=53759.ckpt", unzip=False)

In [2]:
model = TumorSegmentation.load_from_checkpoint("/content/epoch=29-step=53759.ckpt")
model.eval();
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device);

NameError: name 'TumorSegmentation' is not defined

In [ ]:
preds = []
labels = []

for slice, label in tqdm(val_dataset):
    slice = torch.tensor(slice).float().to(device).unsqueeze(0)
    with torch.no_grad():
        pred = torch.sigmoid(model(slice))
    preds.append(pred.cpu().numpy())
    labels.append(label)

preds = np.array(preds)
labels = np.array(labels)

Compute overall Dice Score: This is not a bad result!
Those tumors are extremely small and already some wrongly segmented pixels strongly reduce the Dice Score.
The Visualization below demonstrates that!

In [ ]:
dice_score = DiceScore()(torch.from_numpy(preds), torch.from_numpy(labels).unsqueeze(0).float())
print(f"The Val Dice Score is: {dice_score}")

## Visualization

Feel free to play around with the threshold.

What happens if you decrease it?

In [102]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import Video
from tqdm import tqdm


subject = Path("/content/Task06_Lung/imagesTs/lung_013.nii.gz")
ct2 = nib.load(subject).get_fdata() / 3071  # standardize
ct2 = ct2[:,:,30:]  # crop



THRESHOLD = 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"

segmentation = []
scan = []

# --- predykcje modelu ---
for i in tqdm(range(ct2.shape[-1])):
    slice_ = cv2.resize(ct2[:, :, i], (256, 256))
    scan.append(slice_)
    input_t = torch.tensor(slice_).unsqueeze(0).unsqueeze(0).float().to(device)
    with torch.no_grad():
        pred = torch.sigmoid(model(input_t))[0, 0].cpu().numpy()
    pred = (pred > THRESHOLD).astype(np.uint8)
    segmentation.append(pred)

# --- animacja ---
fig = plt.figure(figsize=(6, 6))
camera = Camera(fig)

for i in range(0, len(scan), 2):
    plt.imshow(scan[i], cmap="bone")
    mask = np.ma.masked_where(segmentation[i] == 0, segmentation[i])
    plt.imshow(mask, alpha=0.5, cmap="autumn")
    plt.axis("off")
    camera.snap()

animation = camera.animate()

# --- zapis (naprawa błędu: brak plt.close) ---
save_path13 = "/content/ct_tumor_animation_13.mp4"
print("💾 Rendering i zapis wideo...")
animation.save(save_path13, writer="ffmpeg", fps=10, dpi=150)
print("✅ Zapisano:", save_path13)


# --- wyświetl ---
Video(save_path13, embed=True)


100%|██████████| 235/235 [00:04<00:00, 55.57it/s]


<IPython.core.display.Javascript object>

💾 Rendering i zapis wideo...
✅ Zapisano: /content/ct_tumor_animation_13.mp4


Plotting the predicted segmentation (red)

In [103]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera
from IPython.display import Video
from tqdm import tqdm


subject = Path("/content/Task06_Lung/imagesTs/lung_035.nii.gz")
ct2 = nib.load(subject).get_fdata() / 3071  # standardize
ct2 = ct2[:,:,30:]  # crop



THRESHOLD = 0.5
device = "cuda" if torch.cuda.is_available() else "cpu"

segmentation = []
scan = []

# --- predykcje modelu ---
for i in tqdm(range(ct2.shape[-1])):
    slice_ = cv2.resize(ct2[:, :, i], (256, 256))
    scan.append(slice_)
    input_t = torch.tensor(slice_).unsqueeze(0).unsqueeze(0).float().to(device)
    with torch.no_grad():
        pred = torch.sigmoid(model(input_t))[0, 0].cpu().numpy()
    pred = (pred > THRESHOLD).astype(np.uint8)
    segmentation.append(pred)

# --- animacja ---
fig = plt.figure(figsize=(6, 6))
camera = Camera(fig)

for i in range(0, len(scan), 2):
    plt.imshow(scan[i], cmap="bone")
    mask = np.ma.masked_where(segmentation[i] == 0, segmentation[i])
    plt.imshow(mask, alpha=0.5, cmap="autumn")
    plt.axis("off")
    camera.snap()

animation = camera.animate()

# --- zapis (naprawa błędu: brak plt.close) ---
save_path35 = "/content/ct_tumor_animation_35.mp4"
print("💾 Rendering i zapis wideo...")
animation.save(save_path35, writer="ffmpeg", fps=10, dpi=150)
print("✅ Zapisano:", save_path35)


# --- wyświetl ---
Video(save_path35, embed=True)


100%|██████████| 292/292 [00:05<00:00, 55.95it/s]


<IPython.core.display.Javascript object>

💾 Rendering i zapis wideo...
✅ Zapisano: /content/ct_tumor_animation_35.mp4


Congratulations! You created a lung cancer segmentation model!